In [2]:
import numpy as np
import gym
import os
from progressbar import ProgressBar
from time import sleep
import scipy
from matplotlib import image
from progressbar import ProgressBar

from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, Flatten
from keras import optimizers
from keras import backend as K

Using TensorFlow backend.


In [3]:
def create_network(action_size, initializer='glorot_uniform'):
    optimizer = optimizers.RMSprop(lr=alpha)
    
    net = Sequential([
        Conv2D(
            32,
            (8, 8),
            strides=(4,4),
            input_shape=(84, 84, 4),
            activation='relu',
            kernel_initializer=initializer,
            data_format="channels_last"
        ),
        Conv2D(
            64,
            (4,4),
            strides=(2,2),
            activation='relu',
            kernel_initializer=initializer
        ),
        Conv2D(
            64,
            (3,3),
            strides=(1,1),
            activation='relu',
            kernel_initializer=initializer
        ),
        Flatten(),
        Dense(512, activation='relu', kernel_initializer=initializer),
        Dense(action_size, activation='linear', kernel_initializer=initializer)
    ])
    net.compile(loss='mse', optimizer=optimizer)
    return net

In [4]:
def pre_process_frame(frame, prev_frame):
    r = frame[:,:,0] # red channel
    g = frame[:,:,1] # blue channel
    b = frame[:,:,2] # green channel
    l = (0.2126*r + 0.7152*g + 0.0722*b) # luminescence 
    max_r = max(np.max(prev_frame[:,:,0]), np.max(r))
    max_g = max(np.max(prev_frame[:,:,1]), np.max(g))
    max_b = max(np.max(prev_frame[:,:,2]), np.max(b))
    
    
    normalized_color = np.zeros((frame.shape[0], frame.shape[1], 4))
    normalized_color[:,:,0] = r.astype(np.float64) / max_r
    normalized_color[:,:,1] = g.astype(np.float64) / max_g
    normalized_color[:,:,2] = b.astype(np.float64) / max_b
    normalized_color[:,:,3] = l
    
    
    phi_frame = np.zeros((84, 84, 4))
    zoom_shape = np.array(phi_frame.shape, dtype=float) / np.array(normalized_color.shape, dtype=float)
    phi_frame = scipy.ndimage.zoom(normalized_color, zoom_shape)
    return phi_frame


In [5]:


alpha = 0.00025
hidden_size=32,
episode_count=1000
action_update_frequency=4, # how often do we train the network
param_update_freq=8, # how often do we train the network
target_network_update_freq=100, # how many steps before target network param update (DDQN only)
report_frequency=100, # how many episodes per yield
sample_size=32, # how many samples do we take each time
replay_size=25000, # what is the max replay memory
replay_start=500, # whhen does training start (replay mem size)

# step increment at begining of loop
step = -1

# trainy boi
training_started = False
replay_memory = []

# stats
training_rewards = []
rewards = []
mean_rewards = []
mean_training_rewards = []
alphas = []
epsilons = []
losses = []

# init env
env = gym.make('Breakout-v0')
state_size = env.observation_space.shape[0]
action_size = env.env.action_space.n

# init networks
q_net = create_network(action_size)
q_net = create_network(action_size, 'zeros')

In [6]:
pbar = ProgressBar()
epsilon = .99


for episode in pbar(range(0, episode_count)):
    done = False
    lives = -1
    frame = env.reset()
    prev_frame = np.zeros_like(frame) # the "prevvious" frame is nothing

    while not done:
        old_lives = lives  
        env.render()
    #     sleep(0.05)

        phi_frame = pre_process_frame(frame, prev_frame)
        phi_shape = phi_frame.shape
        phi_frame = phi_frame.reshape(1, phi_shape[0], phi_shape[1], phi_shape[2])
        
        if np.random.rand() < epsilon:
            action = np.random.randint(action_size)
        else:
            action = np.argmax(q_net.predict(phi_frame))
            
        frame, reward, done, ale_lives = env.step(action)
        print(ale_lives)

        prev_frame = frame

N/A% (0 of 1000) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 

  0% (1 of 1000) |                        | Elapsed Time: 0:00:06 ETA:  1:55:14

{'ale.lives': 0}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 

  0% (2 of 1000) |                        | Elapsed Time: 0:00:13 ETA:  1:56:24

...

  0% (6 of 1000) |                        | Elapsed Time: 0:00:32 ETA:  1:29:41

{'ale.lives': 0}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 

  0% (7 of 1000) |                        | Elapsed Time: 0:00:37 ETA:  1:28:49

{'ale.lives': 1}
{'ale.lives': 1}
{'ale.lives': 1}
{'ale.lives': 0}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 5}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 4}
{'ale.lives': 

KeyboardInterrupt: 